# Make a function to extract and write csv with rois and labels

In [7]:
import os
import nibabel as nib
import numpy as np
import matplotlib as plt
import pandas as pd
from nilearn.input_data import NiftiLabelsMasker
from nilearn import datasets
from nilearn.connectome import ConnectivityMeasure
from nilearn import plotting
from joblib import Parallel, delayed
import multiprocessing
%matplotlib inline

parcellation='power-drysdale'
input_file_list_fname = "../post_processing/rs_files_and_exclusions.csv"
parcel_labels_fname='../power_spheres/power_drysdale_labels.csv'
label_image_fname='../power_spheres/power_drysdale_spheres.nii'

#get our labels -- these are just network labels
input_file_list = pd.read_csv(input_file_list_fname)
label_df = pd.read_csv(parcel_labels_fname)
labels = list(label_df['label'].values)

if 'exclude' in input_file_list.columns:
    exclude = input_file_list.exclude.values
else:
    exclude = [0]*input_file_list.file.shape[0]

masker_obj = NiftiLabelsMasker(labels_img=label_image_fname, standardize=True,
                            memory='nilearn_cache', verbose=5)
connectivity_obj = ConnectivityMeasure(kind='correlation')
     
def time_course_extractor(masker_obj, connectivity_obj):
    def extract(fname):
        if os.path.isfile(rsfilename):
            anImg = nib.load(fname)
            time_series = masker_obj.fit_transform(anImg)
        else:
            time_series = []            
        return time_series
    return extract

pd_extractor = time_course_extractor(masker_obj, connectivity_obj)    

3420

In [ ]:
num_cores = 4
rs_fnames_include = rs_fname_list[rs_fname_list.exclude == 0].iloc[0:9,]
import time

t0 = time.time()
results = Parallel(n_jobs=num_cores)(delayed(extractor_func)(f) for f in rs_fnames_include.file.values)
t1 = time.time()

total = t1-t0

In [ ]:
cormats = correlation_measure.fit_transform(results)

In [ ]:
total/10*3420/60/60